In [1]:
import flopy
import pyemu
import pandas as pd
from flopy import utils as fu
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import platform
if 'window' in platform.platform():
    newln = '\n'
else:
    newln = '\r\n'

In [2]:
if not os.path.exists('junk'):
    os.mkdir('junk')

In [3]:
inmod = flopy.modflow.Modflow.load('freyberg.nam')
inmod.model_ws = 'junk'
dis = inmod.get_package('DIS')



changing model workspace...
   junk


In [4]:
inhyd = inmod.get_package('HYD')
Xall, Yall = inmod.sr.xcentergrid, inmod.sr.ycentergrid

In [5]:
# decode the byte string nonsense if in Python 3
hyddf=pd.DataFrame(inhyd.obsdata)
for cc in hyddf.columns:
    try:
        hyddf[cc] = [i.decode() for i in hyddf[cc]]
    except:
        pass

In [6]:
pst = pyemu.pst.Pst('freyberg.pst')

In [7]:
pstobs = pst.obs_data
pstobs.loc[(pstobs.obgnme == 'head_cal') | (pstobs.obgnme == 'head_fore')]

,obsnme,obsval,weight,obgnme
obsnme,,,,
cr03c16,cr03c16,22.18672,1.0,head_cal
cr03c10,cr03c10,24.08591,1.0,head_cal
cr04c9,cr04c9,22.33339,1.0,head_cal
cr10c2,cr10c2,31.83430,1.0,head_cal
cr14c11,cr14c11,17.72841,1.0,head_cal
cr16c17,cr16c17,16.56648,1.0,head_cal
cr22c11,cr22c11,18.28594,1.0,head_cal
cr23c16,cr23c16,15.20820,1.0,head_cal
cr25c5,cr25c5,24.24294,1.0,head_cal


In [8]:
inbas = inmod.get_package('BAS6')

In [9]:
ib = np.squeeze(inbas.ibound.array)
ib[0,:]  = 0
ib[:,0]  = 0
ib[:,-1] = 0
ib[-1,:] = 0

In [10]:
c,r = np.meshgrid(list(range(inmod.ncol)),list(range(inmod.nrow)))
c[ib!=1] = -99
r[ib!=1] = -99


In [11]:
r_potential_obs = [i[0] for  i in r.reshape(40*20,1)  if i > 0 ]
c_potential_obs = [i[0] for  i in c.reshape(40*20,1)  if i > 0 ]


In [12]:
rc_potential_obs = list(zip(r_potential_obs,c_potential_obs))

In [13]:
potential_obs_names = []

In [14]:
hyddf.head()

,pckg,arr,intyp,klay,xl,yl,hydlbl
0,BAS,HD,I,1,4125.0,9125.0,cr03c16
1,BAS,HD,I,1,2625.0,9125.0,cr03c10
2,BAS,HD,I,1,2375.0,8875.0,cr04c9
3,BAS,HD,I,1,625.0,7375.0,cr10c2
4,BAS,HD,I,1,2875.0,6375.0,cr14c11


In [15]:
for r,c in rc_potential_obs:
    cx = Xall[r,c]
    cy = Yall[r,c]
    hyddf = hyddf.append(pd.Series({'pckg':'BAS',
                              'arr': 'HD',
                              'intyp': 'I',
                              'klay': 1,
                              'xl': cx,
                              'yl': cy,
                              'hydlbl': 'pr{0:02d}c{1:02d}'.format(r,c)}),
                 ignore_index=True)
    potential_obs_names.append('pr{0:02d}c{1:02d}'.format(r,c))

In [16]:
hyddf.head()

,pckg,arr,intyp,klay,xl,yl,hydlbl
0,BAS,HD,I,1,4125.0,9125.0,cr03c16
1,BAS,HD,I,1,2625.0,9125.0,cr03c10
2,BAS,HD,I,1,2375.0,8875.0,cr04c9
3,BAS,HD,I,1,625.0,7375.0,cr10c2
4,BAS,HD,I,1,2875.0,6375.0,cr14c11


In [17]:
hydmod_data = list(map(list,hyddf.values))

In [18]:
# set up HYDMOD in flopy
NHYD = len(hydmod_data)
IHYDUN = 42
HYDNOH = -9999999
hyd = flopy.modflow.ModflowHyd(inmod, nhyd=NHYD, ihydun=IHYDUN, hydnoh=HYDNOH, obsdata=hydmod_data)
inmod.write_input()

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfhyd.ModflowHyd'> <class 'flopy.modflow.mfhyd.ModflowHyd'>
replacing existing Package...


In [19]:
hyddf.tail()

,pckg,arr,intyp,klay,xl,yl,hydlbl
1245,BAS,HD,I,1,3125.0,375.0,pr38c12
1246,BAS,HD,I,1,3375.0,375.0,pr38c13
1247,BAS,HD,I,1,3625.0,375.0,pr38c14
1248,BAS,HD,I,1,3875.0,375.0,pr38c15
1249,BAS,HD,I,1,4125.0,375.0,pr38c16


In [20]:
# make an instruction file for heads
with open('freyberg.heads_potobs.ins', 'w') as ofp:
    ofp.write('pif ~{0}'.format(newln))
    for i,cob in enumerate(hyddf.hydlbl.tolist()):
        if i ==0:
            ofp.write('l2  w !{0}!{1}'.format(cob,newln))
        else:
            ofp.write('l1  w !{0}!{1}'.format(cob,newln))
    

In [30]:
potobsdf = pd.DataFrame({'obsnme':potential_obs_names})

In [31]:
potobsdf['obsval'] = -999999
potobsdf['weight'] = 0.0
potobsdf['obgnme'] = 'potobs'

In [33]:
potobsdf.to_csv('potobs_group.csv', index=None)